In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# # %pip install "torch==2.4.0" 
# %pip install -q tensorboard pillow wandb 
# %pip install "torchvision==0.19.0"
# # Install Hugging Face libraries
# %pip install -q --upgrade \
#   "transformers==4.45.1" \
#   "datasets==3.0.1" \
#   "accelerate==0.34.2" \
#   "evaluate==0.4.3" \
#   "bitsandbytes==0.44.0" \
#   "trl==0.11.1" \
#   "peft==0.13.0" \
#   "qwen-vl-utils"
# %pip install -q pillow -U

## 문제 정의

과제물 사진과 과제물 정보를 주고 실제 정답을 맞추는 모델을 개발합니다. 

이 모델은 선생님들이 학생들의 수행평가를 채점한다고 가정합니다. 
본 프로젝트는 실제 교육 현장의 데이터와 차이가 있을 수 있으나, 자동 채점 시스템의 기본 원리와 가능성을 탐구하는 데 중점을 둡니다.

이번 예시에서는 Ko-SciecneQA 데이터셋을 사용할 건데요. 이 데이터셋은 12,726개의 아마존 제품의 제목, 이미지, 설명 및 메타데이터를 포함하고 있습니다.  
이 중에서 이미지를 가지고 있는 6,218개의 데이터 중 시간 절약을 위해서 모두 사용하지는 않고 여기서 20%(1,243)만 사용하겠습니다.

이미지, 문제, 힌트을 기반으로 정답을 생성하도록 모델을 파인튜닝하려 합니다.  
따라서 이미지, 문제, 힌트를 포함한 입력을 만들고, 이를 이용하여 정답을 찾아보겠습니다. 

In [3]:
# 허브에서 데이터셋 로드
from datasets import load_dataset

dataset = load_dataset("daje/Ko-SciecneQA", split="train")
print(dataset)
dataset = dataset.filter(lambda example: example["image"] is not None)
dataset

/root/anaconda3/envs/science/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution', 'korean_question', 'korean_hint', 'korean_choices', 'answer_str'],
    num_rows: 12726
})


Dataset({
    features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution', 'korean_question', 'korean_hint', 'korean_choices', 'answer_str'],
    num_rows: 6218
})

In [4]:
dataset[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>,
 'question': 'Which of these states is farthest north?',
 'choices': ['West Virginia', 'Louisiana', 'Arizona', 'Oklahoma'],
 'answer': 0,
 'hint': '',
 'task': 'closed choice',
 'grade': 'grade2',
 'subject': 'social science',
 'topic': 'geography',
 'category': 'Geography',
 'skill': 'Read a map: cardinal directions',
 'lecture': 'Maps have four cardinal directions, or main directions. Those directions are north, south, east, and west.\nA compass rose is a set of arrows that point to the cardinal directions. A compass rose usually shows only the first letter of each cardinal direction.\nThe north arrow points to the North Pole. On most maps, north is at the top of the map.',
 'solution': 'To find the answer, look at the compass rose. Look at which way the north arrow is pointing. West Virginia is farthest north.',
 'korean_question': '다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?',
 'korean_hint': '',
 'korean_choices': ['웨스트버지니아

In [5]:
# 참고: 이미지는 프롬프트에 직접 제공되지 않고 "processor"의 일부로 포함됨
prompt= """주어진 이미지와 ##QUESTION##, ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. 

##QUESTION##: {korean_question}
##CHOICES##: {korean_choices}
##HINT##: {korean_hint}"""

system_message = "당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다."

In [6]:

# 데이터셋을 OpenAI 메시지 형식으로 변환하는 함수      
def format_data(sample):
   return {"messages": [
               {
                   # 시스템 역할
                   "role": "system", 
                   # 시스템 메시지
                   "content": [{"type": "text", "text": system_message}], 
               },
               {
                   # 사용자 역할
                   "role": "user",  
                   "content": [
                       {
                           "type": "text",
                           # 제품명과 카테고리를 포함한 프롬프트 생성
                           "text": prompt.format(
                              korean_question=sample["korean_question"], 
                              korean_choices=sample["korean_choices"], 
                              korean_hint=sample["korean_hint"]
                              ),
                       },{
                           # 이미지 타입
                           "type": "image", 
                           # 이미지 파일
                           "image": sample["image"] , 
                       }
                   ],
               },
               {
                   # AI 어시스턴트 역할
                   "role": "assistant", 
                   # 정답 
                   "content": [
                        {
                            "type": "text", 
                            "text": sample["answer_str"]
                        }
                    ], 
               },
           ],
       }


# 데이터셋을 OpenAI 메시지 형식으로 변환
# PIL.Image 타입을 유지하기 위해 리스트 컴프리헨션 사용
dataset = [format_data(sample) for sample in dataset]

In [7]:
len(dataset), dataset[345]

(6218,
 {'messages': [{'role': 'system',
    'content': [{'type': 'text',
      'text': '당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.'}]},
   {'role': 'user',
    'content': [{'type': 'text',
      'text': "주어진 이미지와 ##QUESTION##, ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. \n\n##QUESTION##: 오하이오주의 주도는 어디인가요?\n##CHOICES##: ['신시내티', '링컨', '클리블랜드', '콜럼버스']\n##HINT##: "},
     {'type': 'image',
      'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>}]},
   {'role': 'assistant', 'content': [{'type': 'text', 'text': '콜럼버스'}]}]})

In [8]:
train_dataset = dataset[:int(len(dataset) * 0.9)]
test_dataset = dataset[int(len(dataset) * 0.9):]

In [9]:
len(train_dataset), len(test_dataset)

(5596, 622)

## trl의 SFTTrainer를 이용한 파인 튜닝

`trl`의 SFTTrainer를 사용해 모델을 파인튜닝할 건데요. SFTTrainer는 오픈소스 LLM과 VLM의 지도 파인튜닝을 매우 간단하게 만들어줍니다.  
SFTTrainer는 `transformers` 라이브러리의 `Trainer`를 상속받아서 로깅, 평가, 체크포인트 등 모든 기능을 지원하면서도 추가적인 편의 기능을 제공합니다.

이번 예시에서는 PEFT 기능을 사용할 예정입니다. PEFT 방법으로는 QLoRA를 사용할 건데, 이는 양자화와 LoRA 튜닝을 같이 사용하여 대규모 언어 모델의 메모리 사용량을 줄이는 기술입니다.

* 참고: 멀티모달 입력에 패딩이 필요하기 때문에 Flash Attention은 사용할 수 없습니다.

Qwen 2 VL 72B 모델을 사용할 예정이지만, `model_id` 변수만 바꾸면 Meta AI의 Llama-3.2-11B-Vision, Mistral AI의 Pixtral-12B 등 다른 모델로도 쉽게 교체할 수 있습니다. bitsandbytes를 사용해 모델을 4비트로 양자화할 예정입니다.

* 참고: 모델이 클수록 더 많은 메모리가 필요합니다. 이번 예시에서는 72B 모델을 사용할 예정입니다.

VLM 학습을 위해 LLM, 토크나이저, 프로세서를 올바르게 준비하는 것이 매우 중요합니다. 프로세서는 특수 토큰과 이미지를 입력에 포함시키는 역할을 담당하는 모듈입니다.

In [53]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct" 

# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   # attn_implementation="flash_attention_2",     # 학습시에는 flash attention 2 미지원
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


다음은 QWEN의 템플릿 예시입니다.  

```python
<|im_start|>system
시스템 프롬프트<|im_end|>
<|im_start|>user
사용자의 질문<|im_end|>
<|im_start|>assistant
거대 언어 모델의 답변<|im_end|>
```

멀티모달 QWEN은 이렇게 사용할 겁니다.

```python
<|im_start|>system
시스템 프롬프트<|im_end|>
<|im_start|>user
사용자의 질문<|vision_start|>이미지<|vision_end|><|im_end|>
<|im_start|>assistant
거대 언어 모델의 답변<|im_end|>
```

In [54]:
text = processor.apply_chat_template(
    test_dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.<|im_end|>
<|im_start|>user
주어진 이미지와 ##QUESTION##, ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. 

##QUESTION##: 표시된 식민지의 이름은 무엇인가요?
##CHOICES##: ['매사추세츠', '로드아일랜드', '코네티컷', '뉴햄프셔']
##HINT##: <|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
매사추세츠<|im_end|>



SFTTrainer는 peft와 기본적으로 통합되어 있어 LoraConfig를 만들어서 트레이너에 제공하기만 하면 됩니다.

In [55]:
test_dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': "주어진 이미지와 ##QUESTION##, ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. \n\n##QUESTION##: 휘트니의 실험이 가장 잘 답할 수 있는 질문을 선택하세요.\n##CHOICES##: ['계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?', '유리잔에 있는 물의 양이 계란이 물에 가라앉거나 뜨는 것에 영향을 미치나요?']\n##HINT##: 아래의 글은 실험을 설명합니다. 글을 읽고 아래의 지시를 따르세요.\n\n휘트니는 여섯 개의 유리잔 각각에 4온스의 물을 부었습니다. 휘트니는 세 개의 유리잔에 각각 한 스푼의 소금을 녹였고, 나머지 세 개에는 소금을 넣지 않았습니다. 그런 다음, 휘트니는 한 유리잔에 계란을 넣고 계란이 뜨는지 관찰했습니다. 그녀는 계란을 꺼내어 말렸습니다. 그녀는 다른 다섯 개의 유리잔에서도 이 과정을 반복하며 계란이 뜨는지 기록했습니다. 휘트니는 이 테스트를 두 개의 계란으로 더 반복하여 민물과 소금물에서 계란이 뜨는 횟수를 비교했습니다.\n그림: 소금물에 떠 있는 계란."},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=302x232>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?'}]}]}

In [56]:
from qwen_vl_utils import process_vision_info

# 모델 답변을 생성하는 함수
def generate_description(messages, model, processor):
   # 추론을 위한 준비
   text = processor.apply_chat_template(
       messages, tokenize=False, add_generation_prompt=True
   )
   image_inputs, video_inputs = process_vision_info(messages)
   inputs = processor(
       text=[text],
       images=image_inputs,
       videos=video_inputs,
       padding=True,
       return_tensors="pt",
   )
   inputs = inputs.to(model.device)
   # 추론: 출력 생성
   generated_ids = model.generate(
      **inputs, 
      max_new_tokens=128,
      top_p=1.0, 
      do_sample=True, 
      temperature=0.1
      )
   generated_ids_trimmed = [
      out_ids[len(in_ids) :] 
      for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
   output_text = processor.batch_decode(
       generated_ids_trimmed, 
       skip_special_tokens=True, 
       clean_up_tokenization_spaces=False
   )
   return output_text[0]

from tqdm.auto import tqdm 
no_train_result = [] 

for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"][:2]
    answer = test_dataset[idx]["messages"][2]["content"][0]["text"]
    base_description = generate_description(messages, model, processor)
    no_train_result.append((answer, base_description))

100%|██████████| 622/622 [03:24<00:00,  3.04it/s]


In [65]:
prediction_result = [(temp[0].strip() == temp[1].replace("##ANSWER##: '", "").replace("'", "").strip()) for temp in no_train_result]
base_score = sum(prediction_result) /  len(prediction_result) * 100
print(f"학습 전 정확도 : {base_score:.2f}%") 

학습 전 정확도 : 33.12%


## 학습하기

In [15]:
from peft import LoraConfig

# QLoRA 논문과 Sebastian Raschka의 실험을 기반으로 한 LoRA 설정
peft_config = LoraConfig(
       # 모델 가중치에 LoRA 업데이트를 적용하는 정도를 조절하는 스케일링 계수
       lora_alpha=128,
       # 과적합을 방지하기 위한 드롭아웃 비율 설정
       lora_dropout=0.05,
       # LoRA의 순위(rank) - 저차원 행렬의 차원을 결정
       r=256,
       # 편향(bias) 업데이트 여부 - 'none'은 편향을 업데이트하지 않음
       bias="none",
       # LoRA를 적용할 대상 모듈들 - 트랜스포머 모델의 주요 투영 레이어들
       target_modules=[
           "q_proj",    # Query 투영 레이어
           "up_proj",   # FFN 상향 투영 레이어
           "o_proj",    # Output 투영 레이어
           "k_proj",    # Key 투영 레이어
           "down_proj", # FFN 하향 투영 레이어
           "gate_proj", # FFN 게이트 투영 레이어
           "v_proj"     # Value 투영 레이어
       ],
       # 작업 유형 지정 - 인과적 언어 모델링(다음 토큰 예측)
       task_type="CAUSAL_LM",
)

학습을 시작하기 전에 사용할 하이퍼파라미터(SFTConfig)를 정의하고 입력이 모델에 올바르게 제공되는지 확인해야 합니다.  

텍스트만 사용하는 지도 파인튜닝과 달리 모델에 이미지도 함께 제공해야 하는데요. 이를 위해 입력을 올바르게 포맷팅하고 이미지 특징을 포함하는 커스텀 DataCollator를 만들어야 합니다.  

Qwen2 팀이 제공하는 유틸리티 패키지의 process_vision_info 메서드를 사용할 예정입니다. Llama 3.2 Vision 같은 다른 모델을 사용하는 경우라면, 동일한 방식으로 이미지 정보가 처리되는지 확인해봐야 합니다.

In [25]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

# SFTConfig를 통해 학습 설정을 정의
args = SFTConfig(
    # 학습된 모델과 체크포인트를 저장할 디렉터리 경로 및 리포지토리 ID
    output_dir="qwen2-7b-instruct-homeworks",              
    # 전체 학습 에포크 수 (데이터셋을 몇 번 반복할지 설정)
    num_train_epochs=2,                                     
    # 각 장비(GPU)당 사용될 배치 사이즈 (메모리와 연관됨)
    per_device_train_batch_size=1,                          
    # 경사 누적 스텝 수 (이 횟수만큼 기울기를 누적한 후 업데이트)
    gradient_accumulation_steps=8,                          
    # 메모리 절약을 위한 gradient checkpointing 활성화 (메모리 최적화)
    gradient_checkpointing=True,                            
    # AdamW 옵티마이저 (fused 버전 사용으로 학습 속도 향상)
    optim="adamw_torch_fused",                              
    # 몇 스텝마다 로그를 출력할지 설정 (여기선 5 스텝마다 로그)
    logging_steps=5,                                        
    # 매 에포크마다 체크포인트 저장 설정
    save_strategy="epoch",                                  
    # 학습률 (QLoRA 논문에서 추천된 값 사용)
    learning_rate=1e-5,                                     
    # bfloat16 정밀도 사용 (메모리 절약 및 속도 향상)
    bf16=True,                                              
    # tf32 정밀도 사용 (NVIDIA GPU에서 학습 속도 향상)
    tf32=True,                                              
    # 기울기 클리핑을 위한 최대 기울기 값 (QLoRA 논문에서 추천된 값)
    max_grad_norm=0.3,                                      
    # 학습 초기에 학습률을 점진적으로 올리는 warmup 비율 (QLoRA 논문에서 추천된 값)
    warmup_ratio=0.03,                                      
    # 일정한 학습률 스케줄러 사용 (학습률이 변하지 않음)
    lr_scheduler_type="constant",                           
    # 학습된 모델을 Hugging Face Hub에 푸시할지 여부
    # push_to_hub=True,                                     
    # TensorBoard를 통해 학습 상태를 모니터링
    report_to="tensorboard",                                
    # reentrant gradient checkpointing 설정 (비재진입 방식 사용)
    gradient_checkpointing_kwargs={"use_reentrant": False}, 
    # 데이터셋에서 텍스트 필드를 위한 더미 필드 (collator에서 필요)
    dataset_text_field="",                                  
    # collator에서 데이터셋 전처리를 건너뛰기 위한 설정
    dataset_kwargs={"skip_prepare_dataset": True}           
)

# 불필요한 열 삭제하지 않도록 설정 (학습 중 사용되지 않는 열이라도 유지)
args.remove_unused_columns = False

In [26]:
# 텍스트와 이미지 쌍을 인코딩하기 위한 데이터 collator 함수 정의
def collate_fn(examples):
    # 각 예제에서 텍스트와 이미지를 추출하고, 텍스트는 채팅 템플릿을 적용
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # 텍스트를 토크나이징하고 이미지를 처리하여 일괄 처리(batch) 형태로 변환
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

    # labels로 사용할 input_ids 복사본 생성 후, 패딩 토큰을 -100으로 설정하여 손실 계산 시 무시하도록 함
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  # 패딩 토큰 손실 계산 제외

    # 특정 이미지 토큰 인덱스는 손실 계산에서 무시 (모델에 따라 다름)
    if isinstance(processor, Qwen2VLProcessor):  
        # Qwen2VL 모델의 이미지 토큰 인덱스
        image_tokens = [151652, 151653, 151655]
    else:
        # 다른 모델에서 이미지 토큰 ID를 얻어 손실 계산에서 제외
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    
    # 손실 계산 시 이미지 토큰 인덱스를 무시하도록 설정
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    
    # 배치에 labels 추가 (손실 계산 시 사용)
    batch["labels"] = labels

    return batch

In [27]:
# 단일 예시 확인
example = dataset[0]  # 데이터셋의 첫 번째 아이템
print("단일 예시 데이터:")
print(example)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "주어진 이미지와 ##QUESTION##, ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. \n\n##QUESTION##: 다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?\n##CHOICES##: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']\n##HINT##: "}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7FE2C8D28A10>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '웨스트버지니아'}]}]}


In [28]:
# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 558])
어텐션 마스크 형태: torch.Size([1, 558])
이미지 픽셀 형태: torch.Size([1620, 1176])
레이블 형태: torch.Size([1, 558])


In [29]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        126674,  18411,  63332,  34395, 126674,  18411, 131417, 132526,  16560,
         15235,  21388,  78952,     13, 151645,    198, 151644,    872,    198,
         54330,  31079,  85251,  90667,  21329,  80573,   7704,  52428,    565,
            11,   7704,   1143,     46,  15835,    565,     11,   7704,     39,
          3221,    565,  18411,  63332,  34395,  36055, 132760,  17877, 131417,
        132526,  16560,  69192,    247,  37087, 129392,  40281,  56039,  78952,
            13, 130887, 135968, 126550,  23573,  36055, 132760,  17877, 131417,
        132526,  50302,     13,   4710,    565,  52428,    565,     25, 126844,
         55673,  70943,  56475, 130887, 139963, 132064,  19391,  64521,  45130,
           111,  33704, 139740,  31328,  19969,  35711,   5267,    565,  30498,
         15835,    565,     25,   2509, 144025,  53189,  79004,  21329,  83036,
         52959,    516

In [30]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        126674,  18411,  63332,  34395, 126674,  18411, 131417, 132526,  16560,
         15235,  21388,  78952,     13, 151645,    198, 151644,    872,    198,
         54330,  31079,  85251,  90667,  21329,  80573,   7704,  52428,    565,
            11,   7704,   1143,     46,  15835,    565,     11,   7704,     39,
          3221,    565,  18411,  63332,  34395,  36055, 132760,  17877, 131417,
        132526,  16560,  69192,    247,  37087, 129392,  40281,  56039,  78952,
            13, 130887, 135968, 126550,  23573,  36055, 132760,  17877, 131417,
        132526,  50302,     13,   4710,    565,  52428,    565,     25, 126844,
         55673,  70943,  56475, 130887, 139963, 132064,  19391,  64521,  45130,
           111,  33704, 139740,  31328,  19969,  35711,   5267,    565,  30498,
         15835,    565,     25,   2509, 144025,  53189,  79004,  21329,  83036,
         52959,    51

In [31]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("\n디코딩된 텍스트:")
print(decoded_text)


디코딩된 텍스트:
<|im_start|>system
당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.<|im_end|>
<|im_start|>user
주어진 이미지와 ##QUESTION##, ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. 

##QUESTION##: 다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?
##CHOICES##: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']
##HINT##: <|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|imag

In [32]:
from trl import SFTTrainer

trainer = SFTTrainer(
    # 앞서 로드한 Qwen2-VL 모델
    model=model,                   
    # SFTConfig를 통해 정의한 학습 설정
    args=args,                      
    # 학습에 사용할 데이터셋
    train_dataset=train_dataset,    
    # 데이터 배치 처리를 위한 collator 함수
    data_collator=collate_fn,       
    # 텍스트 필드 지정 (커스텀 collator 사용으로 빈 값)
    dataset_text_field="",          
    # LoRA 파인튜닝 설정
    peft_config=peft_config,        
    # 텍스트 토크나이징을 위한 토크나이저
    tokenizer=processor.tokenizer,  
)

/root/anaconda3/envs/science/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/anaconda3/envs/science/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/root/anaconda3/envs/science/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/root/anaconda3/envs/science/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to t

Trainer 인스턴스의 train() 메서드를 호출하여 모델 학습을 시작합니다.  
이렇게 하면 학습 루프가 시작되고 3 에폭 동안 모델이 학습됩니다. PEFT 방법을 사용하고 있기 때문에 전체 모델이 아닌 조정된 모델 가중치만 저장할 예정입니다.

In [33]:
# 학습 시작, 모델은 자동으로 허브와 출력 디렉토리에 저장됨
trainer.train()

# 모델 저장
trainer.save_model(args.output_dir)

/root/anaconda3/envs/science/lib/python3.11/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
5,2.233700
10,1.936100
15,1.559300
20,1.226600
25,0.960700
30,0.825600
35,0.733200
40,0.718200
45,0.742400
50,0.705100


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
/root/anaconda3/envs/science/lib/python3.11/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


In [34]:
# 메모리 비우기
del model
del processor
del trainer
torch.cuda.empty_cache()

## 인퍼런스

학습이 완료된 후에는 모델을 평가하고 테스트해볼 예정입니다.  

먼저 기본 모델을 불러와서 임의의 아마존 제품에 대한 설명을 생성해보고, 그 다음 Q-LoRA로 조정된 모델을 불러와 같은 제품에 대한 설명을 생성해볼 것입니다.  
마지막으로 더 효율적인 추론을 위해 어댑터를 기본 모델과 병합한 뒤, 동일한 제품에 대해 다시 한 번 추론을 실행해볼 예정입니다.

In [35]:
model_id = "Qwen/Qwen2-VL-7B-Instruct" 

In [36]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
 
# 기본 모델 호출
model = AutoModelForVision2Seq.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
processor = AutoProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.52s/it]


학습을 하지 않은 상태에서 예측을 하면 622개 중 226개를 맞추는 것을 확인할 수 있습니다.   
이번에는 학습한 로라 어댑터를 불어와서 예측해보겠습니다. 

### 학습한 모델로 inference 하기 

In [39]:
# 로라 어댑터가 있는 경로
adapter_path = "/root/trl/qwen2-7b-instruct-homeworks"

In [40]:
# 로라 모델을 불러오기 
model.load_adapter(adapter_path) 

In [67]:
# 1개 예측해보기 
ft_description = generate_description(messages, model, processor)
print(ft_description)

주어진 이미지에서 보이는 주 중에서 동쪽에 있는 곳은 '노스다코타'입니다. 따라서 정답은 '노스다코타'입니다.


In [43]:
# 학습한 모델로 예측하기 
results = [] 
for idx in tqdm(range(len(test_dataset))):
    messages = test_dataset[idx]["messages"][:2]
    ft_description = generate_description(messages, model, processor)
    results.append(ft_description)

100%|██████████| 622/622 [03:10<00:00,  3.27it/s]


In [49]:
answers = [] 
for idx in tqdm(range(len(test_dataset))):
    answers.append(test_dataset[idx]["messages"][2]["content"][0]["text"])

100%|██████████| 622/622 [00:00<00:00, 466867.77it/s]


In [51]:
ft_score = 0 
for result, answer in zip(results, answers):
    if result == answer : 
        ft_score += 1
ft_score 

536

### 결과 비교하기

In [70]:
print(f"학습 전 정확도 : {base_score:.2f}%") 
print(f"학습 후 정답률 : {ft_score/662*100:.2f}%")

학습 전 정확도 : 33.12%
학습 후 정답률 : 80.97%


## 로라 병합 후 저장하기

In [71]:
ll

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 460
-rw-r--r-- 1 root     32 Jan 15 04:58  README.md
-rw-r--r-- 1 root    497 Jan 15 04:58  deepspeed_zero3.yaml
-rw-r--r-- 1 root 233224 Jan 15 04:58 'llm tuning example.ipynb'
drwxr-xr-x 5 root   4096 Jan 15 08:23  qwen2-7b-instruct-homeworks/
-rw-r--r-- 1 root   3451 Jan 15 04:58  sft.py
-rw-r--r-- 1 root   6065 Jan 15 04:58  sft_vlm.py
-rw-r--r-- 1 root  99168 Jan 15 12:25 'vlm tuning example(ko-scienceQA).ipynb'
-rw-r--r-- 1 root 110276 Jan 15 04:58 'vlm tuning example.ipynb'


In [73]:
from peft import PeftModel
from transformers import AutoProcessor, AutoModelForVision2Seq

adapter_path = "./qwen2-7b-instruct-homeworks"  # 학습된 어댑터 경로
base_model_id = "Qwen/Qwen2-VL-7B-Instruct"  # 기본 모델 ID
merged_path = "merged"  # 병합된 모델을 저장할 경로

# 기본 모델 로드
model = AutoModelForVision2Seq.from_pretrained(base_model_id, low_cpu_mem_usage=True)

# 병합된 모델 저장 경로
# LoRA와 기본 모델을 병합하고 저장
peft_model = PeftModel.from_pretrained(model, adapter_path)  # PEFT 모델 로드
merged_model = peft_model.merge_and_unload()  # 모델 병합
merged_model.save_pretrained(merged_path, safe_serialization=True, max_shard_size="5GB")  # 병합된 모델 저장

processor = AutoProcessor.from_pretrained(base_model_id)  # 프로세서 로드
processor.save_pretrained(merged_path)  # 프로세서 저장

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.37it/s]


[]

## 허깅페이스에 모델 업로드하기

In [74]:
# 병합된 모델 불러오기 
local_model_path = "/root/trl/merged"
model = AutoModelForVision2Seq.from_pretrained(local_model_path)
processor = AutoProcessor.from_pretrained(local_model_path)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  4.53it/s]


In [ ]:
# 허깅페이스 로그인 
!huggingface-cli login --token Your_Huggingface_Token

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `huggingface` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `huggingface`


In [ ]:
!huggingface-cli upload-large-folder --repo-type=model daje/Qwen2-VL-7B-instruct-ScienceQA /root/trl/merged